In [12]:
import nltk
import pickle
import argparse
from collections import Counter
from glob import glob
from tqdm import tqdm
nltk.download('punkt')
import json
import time
import numpy as np
class Vocabulary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if word not in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

def build_vocab(json_list, threshold):
    """Build a simple vocabulary wrapper."""
    
    
    counter = Counter()  # 중복 제거

    for i in tqdm(range(len(json_list))):
        with open(json_list[i], 'r', encoding='utf-8-sig') as file:
            data = json.load(file)
        caption = str(data['content']['file']['patch_discription'])
        tokens = nltk.tokenize.word_tokenize(caption.lower())

        counter.update(tokens)

        if (i + 1) % 10000 == 0:
            print("[{}/{}] Tokenized the captions.".format(i + 1, len(json_list)))

    # If the word frequency is less than 'threshold', discard the word
    words = [word for word, cnt in counter.items() if cnt >= threshold]

    # Create a vocab wrapper and add some special tokens
    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    # Add the words to the vocabulary
    for word in words:
        vocab.add_word(word)

    return vocab


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
report_list=['Gastritis with Intestinal Metaplasia Gastric mucosa shows mild chronic inflammation with replacement of normal gastric epithelium by goblet cells.'
             ,'Gastritis without Intestinal Metaplasia Elongated mucosal glands and lamina propria show mild chronic inflammation with no evidence of intestinal metaplasia.'
             ,'Intestinal-Type Adenocarcinoma, >50% Tumor Cells, No Muscularis Propria Visible Carcinoma with glandular differentiation occupies more than 50% of the tissue, and muscle cells are not visible.'
             ,'Intestinal-Type Adenocarcinoma, <50% Tumor Cells, No Muscularis Propria Visible Gland-forming carcinoma cells constitute less than 50% of the tissue, and muscle cells are absent.'
             ,'Intestinal-Type Adenocarcinoma, >50% Tumor Cells, Muscularis Propria Visible More than half of the tissue is composed of cancer cells with glandular morphology, and the muscularis layer is identified.'
             ,'Intestinal-Type Adenocarcinoma, <50% Tumor Cells, Muscularis Propria Visible Less than half of the tissue consists of epithelial malignancy with glandular architecture, and the muscularis layer is present.'
             ,'Diffuse-Type Adenocarcinoma, >50% Tumor Cells, No Muscularis Propria Visible Over 50% of the tissue is occupied by poorly differentiated cancer cells, and muscle cells are absent.'
             ,'Diffuse-Type Adenocarcinoma, <50% Tumor Cells, No Muscularis Propria Visible Less than 50% of the tissue is occupied by poorly cohesive carcinoma cells, and the muscularis layer is visible.'
             ,'Diffuse-Type Adenocarcinoma, >50% Tumor Cells, Muscularis Propria Visible Poorly differentiated cancer cells occupy more than 50% of the tissue, and the muscularis layer is present.'
             ,'Diffuse-Type Adenocarcinoma, <50% Tumor Cells, Muscularis Propria Visible Poorly cohesive epithelial malignancy comprises less than half of the tissue, and muscle cells are identified.'
             ,'Mixed-Type Adenocarcinoma, >50% Tumor Cells, No Muscularis Propria Visible More than 50% of the tissue is composed of both intestinal and diffuse-type cancer cells, and the muscularis layer is absent.'
             ,'Mixed-Type Adenocarcinoma, <50% Tumor Cells, No Muscularis Propria Visible Less than 50% of the tissue is occupied by mixed-type adenocarcinoma, featuring both intestinal and diffuse-type components, and muscle cells are absent.'
             ,'Mixed-Type Adenocarcinoma, >50% Tumor Cells, Muscularis Propria Visible More than 50% of the tissue comprises epithelial malignancy with both glandular and signet-ring cell differentiation, and the muscularis layer is seen.'
             ,'Mixed-Type Adenocarcinoma, <50% Tumor Cells, Muscularis Propria Visible Less than 50% of the tissue consists of mixed-type adenocarcinoma, characterized by both intestinal and diffuse-type components, and muscle cells are visible.']
start_time = time.time()
token_sum=0
print("Start Time:", time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time)))

for i in tqdm(range(len(report_list))):
    
    caption = str(report_list[i])
    tokens = nltk.tokenize.word_tokenize(caption.lower())
    report_list[i]=len(tokens)
end_time = time.time()
print("End Time:", time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(end_time)))
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.2f} seconds")
np.sum(np.array(report_list)*np.array([1250, 1250, 602, 634, 632, 632, 650, 532, 659, 659, 625, 625, 625, 625]))

Start Time: 2024-12-12 15:19:39


100%|██████████| 14/14 [00:00<00:00, 5480.70it/s]

End Time: 2024-12-12 15:19:39
Elapsed Time: 0.00 seconds


np.int64(308417)

In [2]:
json_list=glob('../../data/synth/011.유방암 병리 이미지 및 판독문 합성 데이터/1.데이터/**/2.라벨링데이터/**/*.json')
vocab=build_vocab(json_list, 4)

  1%|          | 109/20000 [00:13<41:41,  7.95it/s]


KeyboardInterrupt: 

12

In [2]:
json_list=glob('../../data/synth/011.유방암 병리 이미지 및 판독문 합성 데이터/1.데이터/**/2.라벨링데이터/**/*.json')
with open(json_list[0], 'r', encoding='utf-8-sig') as file:
    data = json.load(file)


In [3]:
data

{'content': {'clinical': {'tumor_code': 'BRCA',
   'category': 'BRDC',
   'age': None,
   'sex': None,
   'diagnosis': None,
   'tumor_category': 'abnormal'},
  'file': {'file_type': 'S',
   'file_name': 'save_name.png',
   'patch_id': 'save_name',
   'format': 'PNG',
   'mpp': 0.5,
   'filesize': '2214.9765625KB',
   'width': 1024,
   'height': 1024,
   'class': '상피내암',
   'object': [{'type': 'Polygon',
     'label': 'Stroma',
     'coordinate': [[30.0, 0.0],
      [30.0, 1.0],
      [29.0, 2.0],
      [29.0, 3.0],
      [29.0, 4.0],
      [30.0, 4.0],
      [31.0, 5.0],
      [31.0, 6.0],
      [31.0, 7.0],
      [32.0, 8.0],
      [33.0, 9.0],
      [33.0, 10.0],
      [34.0, 11.0],
      [35.0, 11.0],
      [36.0, 12.0],
      [37.0, 11.0],
      [38.0, 10.0],
      [39.0, 9.0],
      [39.0, 8.0],
      [39.0, 7.0],
      [39.0, 6.0],
      [39.0, 5.0],
      [39.0, 4.0],
      [39.0, 3.0],
      [39.0, 2.0],
      [39.0, 1.0],
      [39.0, 0.0],
      [38.0, 0.0],
      [37.0, 0.0